# Introduction

This project uses the United Nation population data to present the dynamic relationship among population, GDP and Life expectancy. 

Data sets are downloaded from: https://population.un.org/wpp/

Downloaded data sets include: 
1. Total Population (Both Sexes) from 1950 to 2100
2. Life expectancy at exact age(years) from 1970 to 2015
3. Gross Domestic Product (GDP) from 1970 to 2015

Data processes include: 
Step 1. Import and print data features, 
Step 2. Clean data sets, merge into one, 
Step 3. Use plotly to build animated bubble chart

# Import functions

In [67]:
import pandas as pd
import numpy as np
import xlrd
from os import listdir
from os.path import isfile, join
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
from plotly.figure_factory import create_table


In [68]:
os.chdir('/Users/jianfengbei/Google Drive/2020 study/portfolio - United Nation population data/data set')


## Define functions

In [69]:
def print_files(my_path_dir):
  '''read filename_list and identify file types, 
     read first 5 rows of each csv data set; 
     and read all spreedsheets in excel file.'''
  filename_list= [f for f in listdir(my_path_dir) if isfile(join(my_path_dir, f))] #?
  type=[]
  for name in filename_list:
      if name.split('.')[1] not in type:
          type += [name.split('.')[1]]
  print('There are {} data types: {}'.format(len(type), str(type)))
  print('------------------------------------')

  csv_filelist = []
  xlsx_filelist = []
  for name in filename_list:
      if name.split('.')[1] == 'csv':
          csv_filelist += [name]
      if name.split('.')[1] == 'xlsx':
          xlsx_filelist += [name]
  print('There are '+ str(len(csv_filelist)) + ' csv files, include: ' + str(csv_filelist))
  print('------------------------------------')
  print('There are '+ str(len(xlsx_filelist)) + ' xlsx files, include: ' + str(xlsx_filelist))

  list_dict = {'csv': csv_filelist, 'xlsx':xlsx_filelist}
  for i in list_dict.keys():
    if i == 'csv': 
        for file in list_dict[i]:
            df=pd.read_csv(file)
            print('In ' + file + ', shape = '+ str(df.shape) + ', the first 5 row looks like this:')
            print(df.head(5))
            print('------------------')
    if i == 'xlsx':
        for file in list_dict[i]: 
            df=pd.ExcelFile(file) # spreedsheet
            print('In ' + file + ', it has the following spreedsheet:' + str(df.sheet_names))
            print('------------------')
# check missing%
# sample.apply(lambda col:sum(col.isnull())/col.size)

In [70]:
def print_structure(df):
    '''print number of duplicates and data structure.'''
    print('There is/are ' + str(df.duplicated().sum()) + ' duplicate record(s)')
    print('data structure is shown below: ')
    print(df.info(verbose=True, null_counts=True))  #to see length and dtypes

# Print data sets' features

In [71]:
#show all the columns - setting
pd.set_option('display.max_columns',None)

## Import the first excel file: Population projection data from 2019 - 2100

In [72]:
pop_projection='UN_PPP2019_Output_PopTot.xlsx'
data_test=pd.ExcelFile(pop_projection).parse('Median')

print(data_test.head(20)) 


   Unnamed: 0 Unnamed: 1                          Unnamed: 2 Unnamed: 3  \
0         NaN        NaN                                 NaN        NaN   
1         NaN        NaN                                 NaN        NaN   
2         NaN        NaN                                 NaN        NaN   
3         NaN        NaN                                 NaN        NaN   
4         NaN        NaN                                 NaN        NaN   
5         NaN        NaN                                 NaN        NaN   
6         NaN        NaN                                 NaN        NaN   
7         NaN        NaN                                 NaN        NaN   
8         NaN        NaN                                 NaN        NaN   
9         NaN        NaN                                 NaN        NaN   
10        NaN        NaN                                 NaN        NaN   
11        NaN        NaN                                 NaN        NaN   
12        NaN        NaN 

It turns out that the first 15 rows are all NaN. 

In [73]:
df_projected=pd.read_excel(pop_projection, sheet_name='Median', skiprows=16, header=0, index_col=None)
print(df_projected)

     Index    Variant Region, subregion, country or area Notes  Unnamed: 4  \
0        1  Median PI                              WORLD   NaN         900   
1        2  Median PI              UN development groups     a        1803   
2        3  Median PI             More developed regions     b         901   
3        4  Median PI             Less developed regions     c         902   
4        5  Median PI          Least developed countries     d         941   
..     ...        ...                                ...   ...         ...   
284    285  Median PI                            Bermuda    14          60   
285    286  Median PI                             Canada   NaN         124   
286    287  Median PI                          Greenland    26         304   
287    288  Median PI          Saint Pierre and Miquelon     2         666   
288    289  Median PI           United States of America    35         840   

                  Type  Unnamed: 6         2020         2025   

# Clean data

In [74]:
print_structure(df_projected)

  #if there is duplicates: df[df.duplicated()==1]

There is/are 0 duplicate record(s)
data structure is shown below: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289 entries, 0 to 288
Data columns (total 24 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Index                               289 non-null    int64 
 1   Variant                             289 non-null    object
 2   Region, subregion, country or area  289 non-null    object
 3   Notes                               82 non-null     object
 4   Unnamed: 4                          289 non-null    int64 
 5   Type                                289 non-null    object
 6   Unnamed: 6                          289 non-null    int64 
 7   2020                                289 non-null    object
 8   2025                                289 non-null    object
 9   2030                                289 non-null    object
 10  2035                                289 non-null    obj

It is concluded that: 
1. Some columns are unnecessary and should be deleted: 'Index','Variant','Notes','Unnamed: 6'

2. Column order should be re-arranged.

3. A 'continent' column should be added. 

4. Column types should be changed: dtype from column '2020' to column '2100' is 'object' which should be changed to 'float64'. 

In [75]:
#1 - drop columns
df_projected.drop(['Index','Variant','Notes','Unnamed: 6'],axis='columns',inplace=True)

#2 - change column names:'Region, subregion, country or area', 'Unnamed: 4' 
df_projected.columns=['country_name', 'country_code', 'Type', 2020, 2025, 2030, 2035, 2040, 2045, 2050, 2055, 2060, 2065, 2070, 2075, 2080, 2085, 2090, 2095, 2100]

#3 - add continent variable
continent_dict={947: 'Africa', 922: 'Asia', 1830: 'Latin America and the Caribbean', 927: 'Oceania', 917: 'Europe', 918: 'Northern America'}
df_projected['continent']=''
for code,country in continent_dict.items():
    df_projected.iloc[df_projected[df_projected['country_code'] == code].index[0], -1]=country

df_projected['continent'].replace('',np.nan,inplace=True)
df_projected['continent'].ffill(inplace=True)



In [76]:
#keep country/area data only
projected_by_country = df_projected[df_projected['Type']=='Country/Area'].drop('Type',axis='columns')
       #reset index
projected_by_country=projected_by_country.reset_index(drop=True)

#4 - convert columns to float type
column=projected_by_country.columns[3:-1]
for name in column:
    projected_by_country[name]=projected_by_country[name].astype('float')
projected_by_country.dtypes


country_name     object
country_code      int64
2020             object
2025            float64
2030            float64
2035            float64
2040            float64
2045            float64
2050            float64
2055            float64
2060            float64
2065            float64
2070            float64
2075            float64
2080            float64
2085            float64
2090            float64
2095            float64
2100            float64
continent        object
dtype: object

In [77]:
#move 'continent' column to after 'country_name'
projected_by_country=projected_by_country[['country_name', 'country_code', 'continent', 2020, 2025, 2030, 2035, 2040, 2045, 2050, 2055, 2060, 2065, 2070, 2075, 2080, 2085, 2090, 2095, 2100]]
print(projected_by_country)


                  country_name  country_code         continent     2020  \
0                      Burundi           108            Africa  11890.8   
1                      Comoros           174            Africa  869.595   
2                     Djibouti           262            Africa  988.002   
3                      Eritrea           232            Africa  3546.43   
4                     Ethiopia           231            Africa   114964   
..                         ...           ...               ...      ...   
230                    Bermuda            60  Northern America   62.273   
231                     Canada           124  Northern America  37742.2   
232                  Greenland           304  Northern America   56.772   
233  Saint Pierre and Miquelon           666  Northern America    5.795   
234   United States of America           840  Northern America   331003   

           2025        2030        2035        2040        2045        2050  \
0     13763.925   15

# Import data: Population data from 1950 - 2015

In [78]:
pop_hist='TotalPopSex-20200508024402.xlsx'
df_hist=pd.read_excel(pop_hist, sheet_name='Data', skiprows=1, header=0, index_col=None)
df_hist.drop(['ISO 3166-1 numeric code','Sex', 'Note'],axis='columns',inplace=True)
       #remove duplicate spaces in 'Location':
df_hist['Location']=df_hist.Location.str.lstrip()
df_hist=df_hist.rename(columns={'Location':'country_name'})

print_structure(df_hist)

There is/are 3 duplicate record(s)
data structure is shown below: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289 entries, 0 to 288
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country_name  289 non-null    object 
 1   1950          285 non-null    float64
 2   1955          285 non-null    float64
 3   1960          285 non-null    float64
 4   1965          285 non-null    float64
 5   1970          285 non-null    float64
 6   1975          285 non-null    float64
 7   1980          285 non-null    float64
 8   1985          285 non-null    float64
 9   1990          285 non-null    float64
 10  1995          285 non-null    float64
 11  2000          285 non-null    float64
 12  2005          285 non-null    float64
 13  2010          285 non-null    float64
 14  2015          285 non-null    float64
dtypes: float64(14), object(1)
memory usage: 34.0+ KB
None


In [79]:
df_hist=df_hist[df_hist.duplicated()==0]

# Merge data

In [80]:
# right join df1950 with projected_by_country:
df_pop=pd.merge(df_hist,projected_by_country,on='country_name')
print(df_pop.notnull().sum())
print(df_pop.columns)
df_pop=df_pop[['country_name','country_code', 'continent',1950,1955,1960,1965,1970,1975,1980,1985, 1990,1995,2000,2005,2010,2015,2020,2025,2030, 2035,2040,2045, 2055,2060,2065,2070, 2075,2080,2085, 2090, 2095,2100]]


country_name    235
1950            235
1955            235
1960            235
1965            235
1970            235
1975            235
1980            235
1985            235
1990            235
1995            235
2000            235
2005            235
2010            235
2015            235
country_code    235
continent       235
2020            235
2025            235
2030            235
2035            235
2040            235
2045            235
2050            235
2055            235
2060            235
2065            235
2070            235
2075            235
2080            235
2085            235
2090            235
2095            235
2100            235
dtype: int64
Index(['country_name',           1950,           1955,           1960,
                 1965,           1970,           1975,           1980,
                 1985,           1990,           1995,           2000,
                 2005,           2010,           2015, 'country_code',
          'continent', 

In [81]:
# melt dataframe to a tidied data
year= [1950,1955,1960,1965,1970,1975,1980,1985, 1990,1995,2000,2005,2010,2015,2020,2025,2030, 2035,2040,2045, 2055,2060,2065,2070, 2075,2080,2085, 2090, 2095,2100]
df_pop=pd.melt(frame=df_pop,id_vars=['country_name','continent'],value_vars=year,var_name='year',value_name='population')
df_pop.sort_values(['country_name','continent','year'],inplace=True)
#reset index
df_pop=df_pop.reset_index(drop=True)
print(df_pop.head())



  country_name continent  year population
0  Afghanistan      Asia  1950       7752
1  Afghanistan      Asia  1955       8271
2  Afghanistan      Asia  1960       8997
3  Afghanistan      Asia  1965       9956
4  Afghanistan      Asia  1970      11174


# Import and merge GDP data

In [82]:
gdp_raw=pd.read_csv('UNdata_GDP_Export_20200530_092907006.csv')
gdp_raw.drop('Item',axis='columns',inplace=True)
gdp_raw.columns=['country_name', 'year','Gross Domestic Product (GDP)']
print_structure(gdp_raw)


There is/are 0 duplicate record(s)
data structure is shown below: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9870 entries, 0 to 9869
Data columns (total 3 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country_name                  9870 non-null   object 
 1   year                          9870 non-null   int64  
 2   Gross Domestic Product (GDP)  9870 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 231.5+ KB
None


In [83]:
df_pop=pd.merge(gdp_raw,df_pop,on=['country_name','year'])
print(df_pop.head())


  country_name  year  Gross Domestic Product (GDP) continent population
0  Afghanistan  2015                    598.835602      Asia      34414
1  Afghanistan  2010                    550.887159      Asia      29186
2  Afghanistan  2005                    258.118123      Asia      25654
3  Afghanistan  2000                    169.965223      Asia      20780
4  Afghanistan  1995                    178.695892      Asia      18111


# Import and merge Life expectancy data

In [84]:
lifeExp_raw=pd.read_csv('UNdata_LifeExp_Export_20200530_092543484.csv')
lifeExp_raw.drop('Variant',axis='columns',inplace=True)
lifeExp_raw.columns=['country_name', 'year','life_expectancy']
lifeExp_raw['year']=lifeExp_raw['year'].str[5:].astype('float')

print_structure(lifeExp_raw)

There is/are 90 duplicate record(s)
data structure is shown below: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8670 entries, 0 to 8669
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   country_name     8670 non-null   object 
 1   year             8670 non-null   float64
 2   life_expectancy  7530 non-null   float64
dtypes: float64(2), object(1)
memory usage: 203.3+ KB
None


In [85]:
lifeExp_raw=lifeExp_raw[lifeExp_raw.duplicated() == 0]

In [86]:
df_pop=pd.merge(lifeExp_raw,df_pop,on=['country_name','year'])
df_pop.to_csv('df_pop.csv')

# Plotting

In [87]:
gapminder = df_pop
gapminder.columns=['country_name', 'year', 'life_expectancy', 'gdp', 'continent', 'population']
table=create_table(gapminder.head(10))
#py.plot(table)


'temp-plot.html'

In [91]:
gapminder.sort_values(['country_name','continent','year'],inplace=True)
gapminder['population']=gapminder['population'].astype('float')
print(gapminder.head())

  country_name  year  life_expectancy         gdp continent  population
9  Afghanistan  1970            36.15  156.518940      Asia     11174.0
8  Afghanistan  1975            38.74  186.510897      Asia     12689.0
7  Afghanistan  1980            41.63  272.655295      Asia     13356.0
6  Afghanistan  1985            44.97  278.257843      Asia     11938.0
5  Afghanistan  1990            48.56  291.774907      Asia     12412.0


In [92]:
#bubble charts: 
fig8=px.scatter(gapminder,x='gdp', y='life_expectancy',color='continent',
           size='population',size_max=60,hover_name='country_name',animation_frame='year',
           animation_group='country_name',log_x=True, range_x=[40,100000],range_y=[25,90],
           labels=dict(population='Population',gdp='GDP per Capita', life_expectancy='Life Expectancy'))
fig8.show()
